In [1]:
import numpy as np
import pandas as pd
from pandas.core.frame import DataFrame

from scipy.interpolate import interp1d

from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import QuantileTransformer, PowerTransformer
from sklearn.dummy import DummyRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns

import xgboost as xgb

from tsfresh.transformers import RelevantFeatureAugmenter, FeatureAugmenter
from tsfresh import extract_relevant_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters, MinimalFCParameters

from typing import Dict, Any
import numpy as np
import pandas as pd
from functools import partial

Function definitions:

In [2]:
def parse_circuit_params_from_str(params_str: str) -> Dict[str, float]:
    return {item.split(":")[0].strip(): float(item.split(":")[1].strip()) for item in params_str.split(",")}

In [3]:
def eis_dataframe_from_csv(csv_path) -> pd.DataFrame:
    """ Reads a CSV file of EIS data into a pandas dataframe

    Args:
        csv_df_path (File-Like-Object): path to file of, or buffer of, EIS data in CSV format

    Returns:
        pd.DataFrame: Dataframe with two or three columns - each row represents an EIS spectrum
            - freq      : short for frequency, column of real-number numpy arrays
            - Z         : short for impedance, column of imaginary-number numpy arrays
            - Circuit   : Equivalent Circuit Model labels assigned to the spectra (Optional)
            - Parameters: Parameters of the Equivalent Circuit Models (Optional)
    """
    df = pd.read_csv(csv_path, index_col=0)

    def real2array(arraystr: str):
        return np.array([float(c.strip("[]")) for c in arraystr.split(", ")])

    def comp2array(arraystr: str):
        return np.array([complex(c.strip("[]").replace(" ", "")) for c in arraystr.split(", ")])

    if "freq" in df.columns:
        df["freq"] = df["freq"].apply(real2array)
    if "Z" in df.columns:
        df["Z"] = df["Z"].apply(comp2array)

    return df

Prior version (interpolated variables)
```
def process_batch_element(freq, impedance, interpolated_basis):
    x_real = np.real(impedance)
    x_imag = -np.imag(impedance)

    f_real = interp1d(freq, x_real)
    f_imag = interp1d(freq, x_imag)
    return np.concatenate((f_real(interpolated_basis), f_imag(interpolated_basis)))

# Separate freq, zreal, zimag
def process_batch_element_f(interpolated_basis):
    return interpolated_basis

def process_batch_element_zreal(freq, Z, interpolated_basis):
    x = np.real(Z)
    f = interp1d(freq, x)
    return f(interpolated_basis)

def process_batch_element_zimag(freq, Z, interpolated_basis):
    x = np.imag(Z)
    f = interp1d(freq, x)
    return f(interpolated_basis)

def process_batch_element_params(Parameters):
    Params = parse_circuit_params_from_str(Parameters)
    return np.array(list(Params.values()))
```



In [4]:
def process_batch_element(freq, impedance):
    x_real = np.real(impedance)
    x_imag = -np.imag(impedance)
    return np.concatenate((x_real, x_imag))

# Separate freq, zreal, zimag
def process_batch_element_f(interpolated_basis):
    return interpolated_basis

def process_batch_element_zreal(freq, Z):
    x = np.real(Z)
    return x

def process_batch_element_zimag(freq, Z):
    x = np.imag(Z)
    f = interp1d(freq, x)
    return x

def process_batch_element_params(Parameters):
    Params = parse_circuit_params_from_str(Parameters)
    return np.array(list(Params.values()))

def process_batch_element_params_str(Parameters):
    Params = parse_circuit_params_from_str(Parameters)
    return np.array(list(Params.keys()))

In [5]:
def unwrap_df(df):
  df2 = DataFrame(columns=['id','freq','zreal','zimag'])
  for i in np.arange(df.shape[0]):
    f, zreal, zimag = df[["f","zreal","zimag"]].loc[i]
    id = np.tile(i, f.size)
    df_ = DataFrame(data=(id,np.log(f),zreal,zimag), index=['id','freq','zreal','zimag']).T
    df2 = df2.append(df_, ignore_index=True)
  return df2

In [6]:
def transform_params(p, mask_skip):
    p_ = np.log(p)
    p_[mask_skip] = p[mask_skip]
    return p_

def invert_params(p, mask_skip):
    p_ = np.exp(p)
    p_[mask_skip] = p[mask_skip]
    return p_

Process data:

In [14]:
df = eis_dataframe_from_csv("data/train_data.csv")
df_test = eis_dataframe_from_csv("data/test_data.csv")

In [16]:
df["f"] = df.apply(lambda x: process_batch_element_f(x.freq), axis=1)
df["zreal"] = df.apply(lambda x: process_batch_element_zreal(x.freq, x.Z), axis=1)
df["zimag"] = df.apply(lambda x: process_batch_element_zimag(x.freq, x.Z), axis=1)
df["param_strs"] = df.apply(lambda x: process_batch_element_params_str(x.Parameters), axis=1)
df["param_values"] = df.apply(lambda x: process_batch_element_params(x.Parameters), axis=1)

df_test["f"] = df_test.apply(lambda x: process_batch_element_f(x.freq), axis=1)
df_test["zreal"] = df_test.apply(lambda x: process_batch_element_zreal(x.freq, x.Z), axis=1)
df_test["zimag"] = df_test.apply(lambda x: process_batch_element_zimag(x.freq, x.Z), axis=1)
df_test["param_strs"] = df_test.apply(lambda x: process_batch_element_params_str(x.Parameters), axis=1)
df_test["param_values"] = df_test.apply(lambda x: process_batch_element_params(x.Parameters), axis=1)


For tsfresh we need to 'unwrap' all the measurements into a dataframe with measurement ids, freq, zreal, and zimag columns

In [17]:
df_ts = unwrap_df(df)
df_ts.shape, len(np.unique(df_ts["id"]))

df_ts_test = unwrap_df(df_test)
df_ts_test.shape, len(np.unique(df_ts_test["id"]))

((91895, 4), 1865)

Train a model for each class of circuit

In [18]:
circuits = np.unique(df["Circuit"])
circuits

array(['L-R-RCPE', 'L-R-RCPE-RCPE', 'L-R-RCPE-RCPE-RCPE', 'RC-G-G',
       'RC-RC-RCPE-RCPE', 'RCPE-RCPE', 'RCPE-RCPE-RCPE',
       'RCPE-RCPE-RCPE-RCPE', 'Rs_Ws'], dtype=object)

In [26]:
dfs = list()
dfs_test = list()
for i in np.arange(len(circuits)):
  dfs.append(df[df["Circuit"] == circuits[i]])
  dfs_test.append(df_test[df_test["Circuit"] == circuits[i]])
dfs[0].index

Int64Index([   9,   27,   29,   30,   32,   38,   42,   44,   47,   49,
            ...
            7389, 7394, 7398, 7403, 7407, 7426, 7440, 7453, 7455, 7460],
           dtype='int64', length=864)

Working model: quantile transform on target.

In [20]:

def MAE_loss(y_true, y_pred):
    return np.mean(np.abs(y_true - y_pred))

def score_qs_vs_pred(y_true, y_pred, model_name):
    results = []
    for i, element in enumerate(df_ecm['param_strs'].iloc[0]):
        # mae_train_element = MAE_loss(y_true_train.apply(lambda x: x[i]), y_pred_train.apply(lambda x: x[i]))
        mae_element = MAE_loss(y_true[:, i], y_pred[:, i])
        res_element = [ecm, element, mae_element]
        results.append(res_element)
    # Save results
    return results

In [21]:
df_ = dfs[0]
df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])

In [33]:
ppls = list()
param_values_pred = list(range(df.shape[0]))
predictions = []
y_true = []
for df_, df_test_ in zip(dfs, dfs_test):
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_ts_test_ = df_ts_test[df_ts_test["id"].isin(df_test[df_test["Circuit"] == df_["Circuit"].loc[df_.index[0]]].index)]
  X_train = pd.DataFrame(index=np.unique(df_ts_["id"]))
  y_train = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_test = pd.DataFrame(index=np.unique(df_ts_test_["id"]))
  y_test = pd.DataFrame(df_test_['param_values'].to_list(), columns=df_test_["param_strs"].loc[df_test_.index[0]])
  # transformer mask, tell the transformer which parameters to skip over
  mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
  mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor())
  regr = TransformedTargetRegressor(regressor=mdl,
                                    transformer=QuantileTransformer(n_quantiles=10, random_state=0))
                                    #transformer=PowerTransformer())
  ppl = Pipeline([
        ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
        ('regressor', regr)
      ])
  ppl.set_params(augmenter__timeseries_container=df_ts_);
  ppl.fit(X_train, y_train)
  ppl.set_params(augmenter__timeseries_container=df_ts_test_);
  y_pred = ppl.predict(X_test)
  y_pred_df = pd.DataFrame(y_pred, columns=df_test_["param_strs"].loc[df_test_.index[0]])
  y_true_df = pd.DataFrame(y_test, columns=df_test_["param_strs"].loc[df_test_.index[0]])
  predictions.append(y_pred_df)
  y_true.append(y_true_df)
  print("MAE:%.4g" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4g" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls.append(ppl)
  for idx, i in zip(df_.index, np.arange(y_pred.shape[0])):
    param_values_pred[idx] = y_pred[i,:]


 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  7.86it/s]


MAE:4240
L1 MAE:1.218e-06
R1 MAE:32.39
R2 MAE:2.117e+04
CPE1_t MAE:0.04722
CPE1_C MAE:0.004859
 
Fitting regression for L-R-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:02<00:00,  8.34it/s]


MAE:4072
L1 MAE:1.216e-06
R1 MAE:38.92
R2 MAE:314.3
CPE1_t MAE:0.1358
CPE1_C MAE:2.001e-06
R3 MAE:3.222e+04
CPE2_t MAE:0.0606
CPE2_C MAE:0.0122
 
Fitting regression for L-R-RCPE-RCPE-RCPE


...

MAE:5960
R1 MAE:10.4
R2 MAE:281
R3 MAE:5.335e+04
CPE1_t MAE:0.1373
CPE1_C MAE:2.474e-06
CPE2_t MAE:0.1106
CPE2_C MAE:5.308e-05
CPE3_t MAE:0.07056
CPE3_C MAE:0.02181
 
Fitting regression for RCPE-RCPE-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  7.85it/s]


MAE:5249
R1 MAE:31.96
R2 MAE:131.4
R3 MAE:2643
R4 MAE:6.018e+04
CPE1_t MAE:0.1356
CPE1_C MAE:4.559e-07
CPE2_t MAE:0.1277
CPE2_C MAE:0.001084
CPE3_t MAE:0.1089
CPE3_C MAE:0.003579
CPE4_t MAE:0.09434
CPE4_C MAE:0.02515
 
Fitting regression for Rs_Ws


Feature Extraction: 100%|██████████| 24/24 [00:01<00:00, 19.14it/s]


MAE:101.8
R1 MAE:23.03
W1_R MAE:191
W1_T MAE:193
W1_p MAE:0.232


In [47]:
def ecm_regression(use_dummy=False, quantile=False, relevant=False):
    ppls = list()
    param_values_pred = list(range(df.shape[0]))
    predictions = []
    y_true = []
    for df_, df_test_ in zip(dfs, dfs_test):
        print(" ")
        print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
        df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
        df_ts_test_ = df_ts_test[df_ts_test["id"].isin(df_test[df_test["Circuit"] == df_["Circuit"].loc[df_.index[0]]].index)]
        X_train = pd.DataFrame(index=np.unique(df_ts_["id"]))
        y_train = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
        X_test = pd.DataFrame(index=np.unique(df_ts_test_["id"]))
        y_test = pd.DataFrame(df_test_['param_values'].to_list(), columns=df_test_["param_strs"].loc[df_test_.index[0]])
        # transformer mask, tell the transformer which parameters to skip over
        mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
        if not use_dummy:
            mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor())
        else: 
            mdl = MultiOutputRegressor(estimator=DummyRegressor(strategy='median'))

        if quantile:
            regr = TransformedTargetRegressor(regressor=mdl,
                                                transformer=QuantileTransformer(n_quantiles=10, random_state=0))
                                                #transformer=PowerTransformer())
        else:
            regr = mdl

        if relevant:
            ppl = Pipeline([
                    ('augmenter', RelevantFeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
                    ('regressor', regr)
                ])
        else:
            ppl = Pipeline([
                    ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
                    ('regressor', regr)
                ])
        ppl.set_params(augmenter__timeseries_container=df_ts_);
        ppl.fit(X_train, y_train)
        ppl.set_params(augmenter__timeseries_container=df_ts_test_);
        y_pred = ppl.predict(X_test)
        y_pred_df = pd.DataFrame(y_pred, columns=df_test_["param_strs"].loc[df_test_.index[0]])
        y_true_df = pd.DataFrame(y_test, columns=df_test_["param_strs"].loc[df_test_.index[0]])
        predictions.append(y_pred_df)
        y_true.append(y_true_df)
        print("MAE:%.4g" % mean_absolute_error(y_test, y_pred))
        p_strs = df_["param_strs"].loc[df_.index[0]]
        for i in np.arange(len(p_strs)):
            print("%s MAE:%.4g" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
        ppls.append(ppl)
        #for idx, i in zip(df_.index, np.arange(y_pred.shape[0])):
            #param_values_pred[idx] = y_pred[i,:]
    return predictions, y_true, ppls #, param_values_pred

In [42]:
predictions_xg, y_true, ppls_xg = ecm_regression(use_dummy=False, quantile=False)

 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  7.46it/s]


MAE:5163
L1 MAE:1.812e-06
R1 MAE:39.35
R2 MAE:2.577e+04
CPE1_t MAE:0.04656
CPE1_C MAE:0.005244
 
Fitting regression for L-R-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  7.93it/s]


MAE:9154
L1 MAE:1.787e-06
R1 MAE:43.77
R2 MAE:332.1
CPE1_t MAE:0.138
CPE1_C MAE:3.569e-06
R3 MAE:7.286e+04
CPE2_t MAE:0.06682
CPE2_C MAE:0.01754
 
Fitting regression for L-R-RCPE-RCPE-RCPE


...

MAE:8379
R1 MAE:10.6
R2 MAE:264.5
R3 MAE:7.514e+04
CPE1_t MAE:0.1362
CPE1_C MAE:2.6e-06
CPE2_t MAE:0.1185
CPE2_C MAE:7.254e-05
CPE3_t MAE:0.06634
CPE3_C MAE:0.02931
 
Fitting regression for RCPE-RCPE-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  7.64it/s]


MAE:1.075e+04
R1 MAE:34.04
R2 MAE:183.7
R3 MAE:2161
R4 MAE:1.266e+05
CPE1_t MAE:0.1388
CPE1_C MAE:0.0002922
CPE2_t MAE:0.1349
CPE2_C MAE:0.002254
CPE3_t MAE:0.1084
CPE3_C MAE:0.006373
CPE4_t MAE:0.09922
CPE4_C MAE:0.03814
 
Fitting regression for Rs_Ws


Feature Extraction: 100%|██████████| 24/24 [00:01<00:00, 18.42it/s]


MAE:112.9
R1 MAE:19.32
W1_R MAE:251.3
W1_T MAE:181
W1_p MAE:0.1415


In [41]:
predictions_xgt, y_true, ppls_xgt = ecm_regression(use_dummy=False, quantile=True)
predictions_d, y_true, ppls_d = ecm_regression(use_dummy=True, quantile=False)

 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  7.66it/s]


MAE:4240
L1 MAE:1.218e-06
R1 MAE:32.39
R2 MAE:2.117e+04
CPE1_t MAE:0.04722
CPE1_C MAE:0.004859
 
Fitting regression for L-R-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  8.06it/s]


MAE:4072
L1 MAE:1.216e-06
R1 MAE:38.92
R2 MAE:314.3
CPE1_t MAE:0.1358
CPE1_C MAE:2.001e-06
R3 MAE:3.222e+04
CPE2_t MAE:0.0606
CPE2_C MAE:0.0122
 
Fitting regression for L-R-RCPE-RCPE-RCPE


...

MAE:8196
R1 MAE:11.95
R2 MAE:328.6
R3 MAE:7.343e+04
CPE1_t MAE:0.1251
CPE1_C MAE:1.741e-06
CPE2_t MAE:0.1273
CPE2_C MAE:5.95e-05
CPE3_t MAE:0.1226
CPE3_C MAE:0.02729
 
Fitting regression for RCPE-RCPE-RCPE-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:13<00:00,  1.82it/s]
/Users/joachim/opt/anaconda3/envs/eis-battmen/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: `n_features_in_` is deprecated in 1.0 and will be removed in 1.2.
  warnings.warn(msg, category=FutureWarning)
Feature Extraction: 100%|██████████| 25/25 [00:03<00:00,  7.12it/s]


MAE:7764
R1 MAE:39.29
R2 MAE:137.5
R3 MAE:2503
R4 MAE:9.049e+04
CPE1_t MAE:0.1298
CPE1_C MAE:4.552e-07
CPE2_t MAE:0.1308
CPE2_C MAE:0.0008804
CPE3_t MAE:0.1294
CPE3_C MAE:0.003501
CPE4_t MAE:0.1297
CPE4_C MAE:0.02773
 
Fitting regression for Rs_Ws


Feature Extraction: 100%|██████████| 25/25 [00:04<00:00,  5.08it/s]
/Users/joachim/opt/anaconda3/envs/eis-battmen/lib/python3.7/site-packages/sklearn/utils/deprecation.py:103: FutureWarning: `n_features_in_` is deprecated in 1.0 and will be removed in 1.2.
  warnings.warn(msg, category=FutureWarning)
Feature Extraction: 100%|██████████| 24/24 [00:01<00:00, 19.34it/s]


MAE:268.2
R1 MAE:100.4
W1_R MAE:783
W1_T MAE:188.6
W1_p MAE:0.9692


In [61]:
y_true[0][col]

0      1.780000e-07
1      1.310000e-07
2      1.160000e-06
3      8.070000e-06
4      5.710000e-06
           ...     
215    4.890000e-07
216    3.190000e-07
217    1.380000e-06
218    5.740000e-06
219    1.270000e-07
Name: L1, Length: 220, dtype: float64

In [69]:
def parse_mae(predictions, y_true, save=False, save_path=None):
    results = []
    for i in range(len(predictions)):
        for j, col in enumerate(predictions[i].columns):
            mae_test_element = MAE_loss(y_true[i][col], predictions[i][col])
            res_element = [i, col, mae_test_element]
            results.append(res_element)
    if save:
        # Save results to csv
        results_df = pd.DataFrame(results, columns=['circuit', 'param', 'mae'])
        results_df.to_csv(save_path+'results.csv')
    return results

In [70]:
results_xg = parse_mae(predictions_xg, y_true, save=True, save_path='xg')
results_xgt = parse_mae(predictions_xgt, y_true, save=True, save_path='xgt')
results_d = parse_mae(predictions_d, y_true, save=True, save_path='d')

In [48]:
predictions_xgt_r, y_true, ppls_xgt_r = ecm_regression(use_dummy=False, quantile=True, relevant=True)


 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 25/25 [00:12<00:00,  2.02it/s]
/Users/joachim/opt/anaconda3/envs/eis-battmen/lib/python3.7/site-packages/tsfresh/utilities/dataframe_functions.py:202: RuntimeWarning: The columns ['zreal__fft_coefficient__attr_"real"__coeff_42'
 'zreal__fft_coefficient__attr_"real"__coeff_43'
 'zreal__fft_coefficient__attr_"real"__coeff_44'
 'zreal__fft_coefficient__attr_"real"__coeff_45'
 'zreal__fft_coefficient__attr_"real"__coeff_46'
 'zreal__fft_coefficient__attr_"real"__coeff_47'
 'zreal__fft_coefficient__attr_"real"__coeff_48'
 'zreal__fft_coefficient__attr_"real"__coeff_49'
 'zreal__fft_coefficient__attr_"real"__coeff_50'
 'zreal__fft_coefficient__attr_"real"__coeff_51'
 'zreal__fft_coefficient__attr_"real"__coeff_52'
 'zreal__fft_coefficient__attr_"real"__coeff_53'
 'zreal__fft_coefficient__attr_"real"__coeff_54'
 'zreal__fft_coefficient__attr_"real"__coeff_55'
 'zreal__fft_coefficient__attr_"real"__coeff_56'
 'zreal__fft_coefficient__attr_"real"__coeff_57'


ValueError: The truth value of a DataFrame is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
predictions_xg_r, y_true, ppls_xg_r = ecm_regression(use_dummy=False, quantile=False, relevant=True)

In [35]:
# Showcase results and compare
for i, df in enumerate(dfs):
  print("Circuit %s" % circuits[i])
  print("MAE:%.4g" % mean_absolute_error(y_true[i], predictions[i]))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for j in np.arange(len(p_strs)):
    print("%s MAE:%.4g" % (p_strs[j], mean_absolute_error(y_true[i][p_strs[j]], predictions[i][p_strs[j]])))

Circuit L-R-RCPE
MAE:4240
R1 MAE:32.39


KeyError: 'W1_R'

In [ ]:
# tsfresh-xgboost no quantile transform
ppls = list()
param_values_pred = list(range(df.shape[0]))
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  # transformer mask, tell the transformer which parameters to skip over
  mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
  mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor())
  #regr = TransformedTargetRegressor(regressor=mdl,
  #                                  transformer=QuantileTransformer(n_quantiles=10, random_state=0))
  #                                  #transformer=PowerTransformer())
  ppl = Pipeline([
        ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
        ('regressor', mdl)
      ])
  ppl.set_params(augmenter__timeseries_container=df_ts_);
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4g" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4g" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls.append(ppl)
  for idx, i in zip(df_.index, np.arange(y_pred.shape[0])):
    param_values_pred[idx] = y_pred[i,:]

 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 1384/1384 [01:36<00:00, 14.37it/s]


[16:24:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:24:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:24:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:24:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:24:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Feature Extraction: 100%|██████████| 346/346 [00:24<00:00, 14.41it/s]


MAE:3836
L1 MAE:1.352e-05
R1 MAE:40.09
R2 MAE:1.914e+04
CPE1_t MAE:0.04905
CPE1_C MAE:0.01479
 
Fitting regression for L-R-RCPE-RCPE


...

MAE:1.144e+04
R1 MAE:33.57
R2 MAE:227.6
R3 MAE:2660
R4 MAE:1.344e+05
CPE1_t MAE:0.1517
CPE1_C MAE:2.1e-05
CPE2_t MAE:0.133
CPE2_C MAE:0.002711
CPE3_t MAE:0.1115
CPE3_C MAE:0.006446
CPE4_t MAE:0.09264
CPE4_C MAE:0.0299
 
Fitting regression for Rs_Ws


Feature Extraction: 100%|██████████| 520/520 [00:36<00:00, 14.09it/s]


[16:44:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:44:45] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:44:47] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[16:44:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Feature Extraction: 100%|██████████| 132/132 [00:09<00:00, 14.32it/s]


MAE:150.5
R1 MAE:41
W1_R MAE:223.3
W1_T MAE:337.5
W1_p MAE:0.189


In [ ]:
# dummy model, no quantile transform
ppls_0 = list()
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  # transformer mask, tell the transformer which parameters to skip over
  mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
  mdl = DummyRegressor(strategy='median')
  #regr = TransformedTargetRegressor(regressor=mdl, 
  #                                  func=partial(transform_params, mask_skip=mask),
  #                                  inverse_func=partial(invert_params, mask_skip=mask))
  ppl = Pipeline([
        ('regressor', mdl)
      ])
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4g" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4g" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls_0.append(ppl)

 
Fitting regression for L-R-RCPE
MAE:7342
L1 MAE:1.372e-06
R1 MAE:41.88
R2 MAE:3.667e+04
CPE1_t MAE:0.1324
CPE1_C MAE:0.01403
 
Fitting regression for L-R-RCPE-RCPE
MAE:6860
L1 MAE:1.671e-06
R1 MAE:65.77
R2 MAE:290.9
CPE1_t MAE:0.1266
CPE1_C MAE:3.477e-06
R3 MAE:5.452e+04
CPE2_t MAE:0.136
CPE2_C MAE:0.01115
 
Fitting regression for L-R-RCPE-RCPE-RCPE
MAE:6933
L1 MAE:1.23e-06
R1 MAE:52.71
R2 MAE:336.5
CPE1_t MAE:0.1257
CPE1_C MAE:8.535e-07
R3 MAE:2181
CPE2_t MAE:0.1226
CPE2_C MAE:0.0003276
R4 MAE:7.369e+04
CPE3_t MAE:0.1193
CPE3_C MAE:0.01829
 
Fitting regression for RC-G-G
MAE:22.4
R1 MAE:74.1
C1 MAE:4.985e-06
R_g1 MAE:1.283
t_g1 MAE:0.7691
R_g2 MAE:45.08
t_g2 MAE:13.17
 
Fitting regression for RC-RC-RCPE-RCPE
MAE:5967
R1 MAE:33.19
R2 MAE:6.394
R3 MAE:40.62
R4 MAE:5.959e+04
C2 MAE:1.216e-05
CPE3_C MAE:0.0001672
CPE4_t MAE:0.1629
CPE4_C MAE:0.5516
C1 MAE:0
CPE3_t MAE:0
 
Fitting regression for RCPE-RCPE
MAE:4632
R1 MAE:50.85
R2 MAE:2.774e+04
CPE1_t MAE:0.1176
CPE1_C MAE:0.0009411
CPE2_

In [ ]:
# dummy model, quantile transform
ppls_0 = list()
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  # transformer mask, tell the transformer which parameters to skip over
  mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
  mdl = DummyRegressor(strategy='median')
  regr = TransformedTargetRegressor(regressor=mdl,
                                    transformer=QuantileTransformer(n_quantiles=10, random_state=0))
  ppl = Pipeline([
        ('regressor', regr)
      ])
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4g" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4g" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls_0.append(ppl)

 
Fitting regression for L-R-RCPE
MAE:7342
L1 MAE:1.372e-06
R1 MAE:41.88
R2 MAE:3.667e+04
CPE1_t MAE:0.1324
CPE1_C MAE:0.01403
 
Fitting regression for L-R-RCPE-RCPE
MAE:6860
L1 MAE:1.671e-06
R1 MAE:65.77
R2 MAE:290.9
CPE1_t MAE:0.1266
CPE1_C MAE:3.477e-06
R3 MAE:5.452e+04
CPE2_t MAE:0.136
CPE2_C MAE:0.01115
 
Fitting regression for L-R-RCPE-RCPE-RCPE
MAE:6933
L1 MAE:1.23e-06
R1 MAE:52.71
R2 MAE:336.5
CPE1_t MAE:0.1257
CPE1_C MAE:8.535e-07
R3 MAE:2181
CPE2_t MAE:0.1226
CPE2_C MAE:0.0003276
R4 MAE:7.369e+04
CPE3_t MAE:0.1193
CPE3_C MAE:0.01829
 
Fitting regression for RC-G-G
MAE:22.4
R1 MAE:74.1
C1 MAE:4.985e-06
R_g1 MAE:1.283
t_g1 MAE:0.7691
R_g2 MAE:45.08
t_g2 MAE:13.17
 
Fitting regression for RC-RC-RCPE-RCPE
MAE:5967
R1 MAE:33.19
R2 MAE:6.394
R3 MAE:40.62
R4 MAE:5.959e+04
C2 MAE:1.216e-05
CPE3_C MAE:0.0001672
CPE4_t MAE:0.1629
CPE4_C MAE:0.5516
C1 MAE:0
CPE3_t MAE:0
 
Fitting regression for RCPE-RCPE
MAE:4632
R1 MAE:50.85
R2 MAE:2.774e+04
CPE1_t MAE:0.1176
CPE1_C MAE:0.0009411
CPE2_

Model trained on all data for submission.

In [ ]:
#train tsfresh-xgboost-quantile models on all data
ppls = list()
param_values_pred = list(range(df.shape[0]))
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor())
  regr = TransformedTargetRegressor(regressor=mdl,
                                    transformer=QuantileTransformer(n_quantiles=10, random_state=0))
  ppl = Pipeline([
        ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
        ('regressor', regr)
      ])
  ppl.set_params(augmenter__timeseries_container=df_ts_);
  ppl.fit(df_x, df_y)
  ppls.append(ppl)

  ####### help
  # save fits
  for idx, i in zip(df_.index, np.arange(y_pred.shape[0])):
    param_values_pred[idx] = y_pred[i,:]


#pickle each model in ppls

#import pickle
#for mdl in ppls:
#  file = pickle.dump(mdl)

 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 1730/1730 [02:24<00:00, 12.00it/s]


[03:57:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:57:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:57:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:57:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[03:57:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
 
Fitting regression for L-R-RCPE-RCPE


Feature Extraction: 100%|██████████| 1800/1800 [02:32<00:00, 11.80it/s]


[04:00:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:00:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:00:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:00:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:00:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:00:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:00:52] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:00:57] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

...

[04:17:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:17:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:17:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:17:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:17:24] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:17:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:17:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:17:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

Feature Extraction: 100%|██████████| 1848/1848 [02:32<00:00, 12.11it/s]


[04:20:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:20:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:20:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:20:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:20:50] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:20:55] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:21:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:21:07] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

Feature Extraction: 100%|██████████| 652/652 [00:53<00:00, 12.17it/s]


[04:22:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:22:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:22:34] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[04:22:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Export df to csv

In [ ]:
param_values_pred

[array([4.2765201e+02, 2.8244915e+00, 4.3445881e-03, 4.3264108e+00],
       dtype=float32),
 array([4.2765201e+02, 2.8244915e+00, 4.3445881e-03, 4.3264108e+00],
       dtype=float32),
 array([0.17008224, 0.14053969, 0.03647374, 1.850966  ], dtype=float32),
 array([4.2765201e+02, 2.8244915e+00, 4.3445881e-03, 4.3264108e+00],
       dtype=float32),
 array([4.2765201e+02, 2.8244915e+00, 4.3445881e-03, 4.3264108e+00],
       dtype=float32),
 array([4.2765201e+02, 2.8244915e+00, 4.3445881e-03, 4.3264108e+00],
       dtype=float32),
 array([4.2765201e+02, 2.8244915e+00, 4.3445881e-03, 4.3264108e+00],
       dtype=float32),
 array([  4.6423497, 493.56506  ,   3.6793091,   2.0064642], dtype=float32),
 array([0.17008224, 0.14053969, 0.03647374, 1.850966  ], dtype=float32),
 array([0.17008224, 0.14053969, 0.03647374, 1.850966  ], dtype=float32),
 array([4.2765201e+02, 2.8244915e+00, 4.3445881e-03, 4.3264108e+00],
       dtype=float32),
 array([0.17008224, 0.14053969, 0.03647374, 1.850966  ], dty

In [ ]:
def write_param_str(param_values, param_strs):
    pstr = ""
    for i in np.arange(len(param_values)):
      if i == 0:
        pstr += param_strs[i] + ": %0.2g" % (param_values[i])
      else:
        pstr += ", " + param_strs[i] + ": %0.2g" % (param_values[i])
    return pstr

In [ ]:
df["param_values_pred"] = param_values_pred
df_ = pd.DataFrame(data=df["Circuit"])
df_["Parameters"] = df.apply(lambda x: write_param_str(x.param_values_pred, x.param_strs), axis=1)

df_

TypeError: ignored

In [ ]:
df_.to_csv('submission.csv')

Other attempts that don't work.
Custom log transform:

In [ ]:
ppls = list()
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  # transformer mask, tell the transformer which parameters to skip over
  mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
  mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor())
  regr = TransformedTargetRegressor(regressor=mdl, 
                                    func=partial(transform_params, mask_skip=mask),
                                    inverse_func=partial(invert_params, mask_skip=mask))
  ppl = Pipeline([
        ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
        ('regressor', regr)
      ])
  ppl.set_params(augmenter__timeseries_container=df_ts_);
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4f" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4f" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls.append(ppl)


 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 1384/1384 [01:56<00:00, 11.86it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


[01:43:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:43:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:43:38] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:43:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:43:46] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Feature Extraction: 100%|██████████| 346/346 [00:27<00:00, 12.43it/s]


MAPE:634548.4890
L1 MAPE:3168187.4009
R1 MAPE:18.5833
R2 MAPE:0.8513
CPE1_t MAPE:0.0857
CPE1_C MAPE:4535.5240
 
Fitting regression for L-R-RCPE-RCPE


Feature Extraction: 100%|██████████| 1440/1440 [02:18<00:00, 10.36it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


[01:46:39] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:49] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:46:56] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:01] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:09] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:47:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

Feature Extraction: 100%|██████████| 360/360 [00:39<00:00,  9.21it/s]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


ValueError: ignored

dummy regressor

In [ ]:
ppls_0 = list()
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  # transformer mask, tell the transformer which parameters to skip over
  mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
  mdl = DummyRegressor(strategy='median')
  regr = TransformedTargetRegressor(regressor=mdl, 
                                    func=partial(transform_params, mask_skip=mask),
                                    inverse_func=partial(invert_params, mask_skip=mask))
  ppl = Pipeline([
        ('regressor', regr)
      ])
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4f" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4f" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls_0.append(ppl)


 
Fitting regression for L-R-RCPE
MAE:7342.5811
L1 MAE:0.0869
R1 MAE:41.8808
R2 MAE:36670.7032
CPE1_t MAE:0.1384
CPE1_C MAE:0.0962
 
Fitting regression for L-R-RCPE-RCPE
MAE:6860.2577
L1 MAE:0.1662
R1 MAE:65.7694
R2 MAE:291.0060
CPE1_t MAE:0.1368
CPE1_C MAE:0.2130
R3 MAE:54524.5107
CPE2_t MAE:0.1459
CPE2_C MAE:0.1139
 
Fitting regression for L-R-RCPE-RCPE-RCPE
MAE:6932.6560
L1 MAE:0.2596
R1 MAE:52.6981
R2 MAE:336.4982
CPE1_t MAE:0.1423
CPE1_C MAE:0.3262
R3 MAE:2181.0637
CPE2_t MAE:0.1379
CPE2_C MAE:0.1955
R4 MAE:73687.6291
CPE3_t MAE:0.1368
CPE3_C MAE:0.1281
 
Fitting regression for RC-G-G
MAE:22.4008
R1 MAE:74.0982
C1 MAE:0.0000
R_g1 MAE:1.2834
t_g1 MAE:0.7691
R_g2 MAE:45.0847
t_g2 MAE:13.1691
 
Fitting regression for RC-RC-RCPE-RCPE
MAE:5966.9245
R1 MAE:33.1936
R2 MAE:6.4065
R3 MAE:40.6432
R4 MAE:59587.7611
C2 MAE:0.1398
CPE3_C MAE:0.1100
CPE4_t MAE:0.1712
CPE4_C MAE:0.5881
C1 MAE:0.2201
CPE3_t MAE:0.0110
 
Fitting regression for RCPE-RCPE
MAE:4631.5568
R1 MAE:50.8647
R2 MAE:27737.88

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-pa

MAE:6428.8155
R1 MAE:12.9561
R2 MAE:381.7258
R3 MAE:57463.5617
CPE1_t MAE:0.1396
CPE1_C MAE:0.3217
CPE2_t MAE:0.1399
CPE2_C MAE:0.2023
CPE3_t MAE:0.1428
CPE3_C MAE:0.1491
 
Fitting regression for RCPE-RCPE-RCPE-RCPE
MAE:8519.5391
R1 MAE:31.9382
R2 MAE:168.4325
R3 MAE:2515.8436
R4 MAE:99516.5397
CPE1_t MAE:0.1628
CPE1_C MAE:0.4466
CPE2_t MAE:0.1436
CPE2_C MAE:0.2983
CPE3_t MAE:0.1432
CPE3_C MAE:0.2106
CPE4_t MAE:0.1466
CPE4_C MAE:0.1634
 
Fitting regression for Rs_Ws
MAE:206.5543
R1 MAE:56.6867
W1_R MAE:674.5952
W1_T MAE:94.0620
W1_p MAE:0.8733


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: RuntimeWarning: overflow encountered in exp
  import sys


wihtout log transform

In [ ]:
ppls_2 = list()
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor())
  ppl = Pipeline([
        ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
        ('regressor', mdl)
      ])
  ppl.set_params(augmenter__timeseries_container=df_ts_);
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4f" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4f" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls_2.append(ppl)

 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 1384/1384 [01:54<00:00, 12.11it/s]


[01:57:04] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:57:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:57:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:57:14] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[01:57:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Feature Extraction: 100%|██████████| 346/346 [00:28<00:00, 12.29it/s]


MAE:3835.6250
L1 MAE:0.0000
R1 MAE:40.0877
R2 MAE:19137.9736
CPE1_t MAE:0.0490
CPE1_C MAE:0.0148
 
Fitting regression for L-R-RCPE-RCPE


...

MAE:11440.2930
R1 MAE:33.5742
R2 MAE:227.6342
R3 MAE:2660.1847
R4 MAE:134361.5944
CPE1_t MAE:0.1517
CPE1_C MAE:0.0000
CPE2_t MAE:0.1330
CPE2_C MAE:0.0027
CPE3_t MAE:0.1115
CPE3_C MAE:0.0064
CPE4_t MAE:0.0926
CPE4_C MAE:0.0299
 
Fitting regression for Rs_Ws


Feature Extraction: 100%|██████████| 520/520 [00:47<00:00, 11.00it/s]


[02:20:23] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:20:25] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:20:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:20:29] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Feature Extraction: 100%|██████████| 132/132 [00:10<00:00, 12.01it/s]


MAE:150.4973
R1 MAE:40.9954
W1_R MAE:223.3179
W1_T MAE:337.4870
W1_p MAE:0.1890


quantile transform on target

In [ ]:
ppls = list()
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  # transformer mask, tell the transformer which parameters to skip over
  mask = [n for n, x in enumerate(df_["param_strs"].loc[df_.index[0]]) if '_t' in x]
  mdl = MultiOutputRegressor(estimator=xgb.XGBRegressor())
  regr = TransformedTargetRegressor(regressor=mdl,
                                    transformer=QuantileTransformer(n_quantiles=10, random_state=0))
                                    #transformer=PowerTransformer())
  ppl = Pipeline([
        ('augmenter', FeatureAugmenter(column_id='id', column_sort='freq', default_fc_parameters=ComprehensiveFCParameters())),
        ('regressor', regr)
      ])
  ppl.set_params(augmenter__timeseries_container=df_ts_);
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4f" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4f" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls.append(ppl)


 
Fitting regression for L-R-RCPE


Feature Extraction: 100%|██████████| 1384/1384 [01:54<00:00, 12.13it/s]


[02:27:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:27:21] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:27:26] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:27:30] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:27:35] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Feature Extraction: 100%|██████████| 346/346 [00:28<00:00, 12.27it/s]


MAE:4745.8882
L1 MAE:0.0000
R1 MAE:34.5926
R2 MAE:23694.7899
CPE1_t MAE:0.0489
CPE1_C MAE:0.0096
 
Fitting regression for L-R-RCPE-RCPE


...

MAE:8039.6460
R1 MAE:30.8346
R2 MAE:161.8707
R3 MAE:2368.0198
R4 MAE:93914.5137
CPE1_t MAE:0.1505
CPE1_C MAE:0.0000
CPE2_t MAE:0.1298
CPE2_C MAE:0.0012
CPE3_t MAE:0.1138
CPE3_C MAE:0.0051
CPE4_t MAE:0.0915
CPE4_C MAE:0.0216
 
Fitting regression for Rs_Ws


Feature Extraction: 100%|██████████| 520/520 [00:42<00:00, 12.29it/s]


[02:51:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:51:15] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:51:16] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:51:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


Feature Extraction: 100%|██████████| 132/132 [00:11<00:00, 11.55it/s]


MAE:105.7937
R1 MAE:21.2443
W1_R MAE:236.3289
W1_T MAE:165.3921
W1_p MAE:0.2096


dummy regressor no transform

In [ ]:
ppls_0 = list()
for df_ in dfs:
  print(" ")
  print("Fitting regression for %s" % df_["Circuit"].loc[df_.index[0]])
  df_ts_ = df_ts[df_ts["id"].isin(df_.index)]
  df_x = pd.DataFrame(index=np.unique(df_ts_["id"]))
  df_y = pd.DataFrame(df_['param_values'].to_list(), columns=df_["param_strs"].loc[df_.index[0]])
  X_train, X_test, y_train, y_test = train_test_split(df_x, df_y, test_size=0.2, random_state=42)
  mdl = DummyRegressor(strategy='median')
  ppl = Pipeline([
        ('regressor', mdl)
      ])
  ppl.fit(X_train, y_train)
  y_pred = ppl.predict(X_test)
  print("MAE:%.4f" % mean_absolute_error(y_test, y_pred))
  p_strs = df_["param_strs"].loc[df_.index[0]]
  for i in np.arange(len(p_strs)):
    print("%s MAE:%.4f" % (p_strs[i], mean_absolute_error(y_test[p_strs[i]], y_pred[:,i])))
  ppls_0.append(ppl)


 
Fitting regression for L-R-RCPE
MAE:7342.4158
L1 MAE:0.0000
R1 MAE:41.8812
R2 MAE:36670.0515
CPE1_t MAE:0.1324
CPE1_C MAE:0.0140
 
Fitting regression for L-R-RCPE-RCPE
MAE:6860.0513
L1 MAE:0.0000
R1 MAE:65.7694
R2 MAE:290.9311
CPE1_t MAE:0.1266
CPE1_C MAE:0.0000
R3 MAE:54523.4364
CPE2_t MAE:0.1360
CPE2_C MAE:0.0112
 
Fitting regression for L-R-RCPE-RCPE-RCPE
MAE:6933.0433
L1 MAE:0.0000
R1 MAE:52.7099
R2 MAE:336.4940
CPE1_t MAE:0.1257
CPE1_C MAE:0.0000
R3 MAE:2180.7010
CPE2_t MAE:0.1226
CPE2_C MAE:0.0003
R4 MAE:73693.1854
CPE3_t MAE:0.1193
CPE3_C MAE:0.0183
 
Fitting regression for RC-G-G
MAE:22.4008
R1 MAE:74.0982
C1 MAE:0.0000
R_g1 MAE:1.2834
t_g1 MAE:0.7691
R_g2 MAE:45.0847
t_g2 MAE:13.1691
 
Fitting regression for RC-RC-RCPE-RCPE
MAE:5966.7458
R1 MAE:33.1870
R2 MAE:6.3942
R3 MAE:40.6226
R4 MAE:59586.5399
C2 MAE:0.0000
CPE3_C MAE:0.0002
CPE4_t MAE:0.1629
CPE4_C MAE:0.5516
C1 MAE:0.0000
CPE3_t MAE:0.0000
 
Fitting regression for RCPE-RCPE
MAE:4632.0036
R1 MAE:50.8451
R2 MAE:27740.92

exploration